In [31]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import sympy.ntheory as nt
from fractions import Fraction
import scipy
import scipy.sparse

In [50]:
def rsagenvals(p, q):
    n = p * q
    f = (p-1) * (q-1)
    k = f - 1
    d = 0
    while (d * k % n != 1):
        d += 1
    return n,k,d
def encode(m, n, k):
    return (m**k)%n
def decode(c,d):
    return (c ** k) % n
    

In [52]:
n, k, d = rsagenvals(3, 7)
m = 5
c = encode(m, n, k)
print(decode(c, d))

5


In [32]:
def StateToVec(myState):
    state = myState[0]
    wires = len(state[1])
    vector = np.zeros(2**wires, dtype = "complex")
    for state in myState:
        decimal = int(int(state[1], 2))
        vector[decimal] += state[0]
    return vector
def VecToState(myVector):
    myState = []
    wires = np.log(len(myVector))/np.log(2)
    for i in range(len(myVector)):
        if myVector[i] != 0:
            binary = bin(i)[2:]
            while (len(binary) < wires):
                binary = "0" + binary
            tuple = (myVector[i], binary)
            myState.append(tuple)
    return myState

In [33]:
def getVector(name, wireCount): 
    file = open(name, "r")
    myVector = np.zeros(2**wireCount, dtype = "complex")
    lines = file.readlines()
    for i in range(len(lines)):
        split = lines[i].split()
        myVector[i] = complex(float(split[0]), float(split[1]))
    file.close()
    return myVector
def ParseInfo(circuitfile):
    file = open(circuitfile, "r")
    lines = file.readlines()
    lineCount = 0
    wireCount = 0
    for i in range(len(lines)):
        if i == 0:
            wireCount = int(lines[i])
            myVector = np.zeros(2**wireCount, dtype = "complex") #likely a placeholder but in case nothing is input
            myVector[0] = 1
            matrix = np.zeros((2**wireCount, 2**wireCount), dtype = "complex")
            for i in range(2**wireCount):
                matrix[i][i] = 1
        else:
            if lines[i][:1] == "H":
                matrix = np.matmul(transformH(lines[i][2:3], wireCount), matrix)
            elif lines[i][:1] == "P":
                matrix = np.matmul(transformP(lines[i][2:3], lines[i][4:], wireCount), matrix)
            elif lines[i][:1] == "C":
                matrix = np.matmul(transformCNOT(lines[i][5:6], lines[i][7:8], wireCount), matrix)
            elif lines[i][:1] == "I": # this entire section is if a state is input
                if lines[i][10:11] == "F":
                    myVector = getVector (lines[i][15:-1], wireCount) #-1 for pesky /n
                elif lines[i][10:11] == "B":
                    myState = [(1,word[17:][:-1])] #-1 to snip >
                    myVector = StateToVec(myState)             
            elif lines[i][:1] == "M":
                #prob is prop to abs val of magnitudea
                myVector = np.matmul(matrix, myVector)
                adVector = abs(myVector)
                adVector = adVector**2
                myVector = np.zeros(2**wireCount)
                for j in range(500):
                    val = random.random()
                    i = 0
                    while (val - adVector[i] > 0):
                        val = val - adVector[i]
                        i += 1
                    myVector[i] += 1
                return myVector
        #print(matrix[0])
    myVector = np.matmul(matrix, myVector)
    file.close()
    return myVector

In [34]:
def factorTwo(N):
    numFactors = 0
    while(N % 2 == 0):
        numFactors += 1
        N *= (1/2)
    return numFactors
def testPowers(N):
    for i in range(2,N-1):
        if(math.log(N,i).is_integer()):
            return i
    return 1
def bashgcd(N):
    val= N
    while math.gcd(val, N) != 1 :
        val = int(random.random() * (N-2))+2
    return val

In [47]:
def HadamardII(inWire,numWires,inputState):
    #|0> -> 1/2 |0> + 1/2 |1> and |1> -> 1/2 |0> - 1/2 |1>
    wires = int(numWires)
    inWire = int(inWire)
    adjust = []
    for line in inputState:
        num = int(int(line[1],2)) #set to 0 wire later
        num2 = num # set to 1 wire later
        val = line[0] * np.sqrt(0.5) #val of 0 wire
        val2 = line[0] * np.sqrt(0.5)#val of 1 wire
        if (num % 2**(inWire+1) >= 2**(inWire)): #input 1 state
            num += -2**(inWire)
            val2 *= -1
        else:
            num2 += 2**(inWire)
        binary = bin(num)[2:]
        
        while (len(binary) < wires):
            binary = "0" + binary
        tuple = (val, binary)
        adjust.append(tuple)
        binary2 = bin(num2)[2:]
        while (len(binary2) < wires):
            binary2 = "0" + binary2
        tuple = (val2, binary2)
        adjust.append(tuple)
    return adjust
def PII(inWire, rads, numWires,inputState):
    wires = int(numWires)
    inWire = int(inWire)
    rads = float(rads)
    adjust = []
    for line in inputState:
        num = int(int(line[1],2))
        if (num % 2**(inWire+1) >= 2**(inWire)): # if 1 input
            val = line[0] *  math.e ** complex(0,rads)
            binary = bin(num)[2:]
            while (len(binary) < wires):
                binary = "0" + binary
            tuple = (val, binary)
            adjust.append(tuple)
        else: #0 input does nothing
            adjust.append(line)
    return adjust
def CNOTII(controlWire, otherWire, numWires, inputState):
    wires = int(numWires)
    cWire = int(controlWire)
    oWire = int(otherWire)
    adjust = []
    for line in inputState:
        num = int(int(line[1],2))
        if (num % 2**(cWire+1) >= 2**(cWire)): # if 1 C input
            val = line[0]
            if (num % 2**(oWire+1) >= 2**(oWire)): #if other 1
                num += -2**(oWire)
            else:
                num += 2**(oWire)
            binary = bin(num)[2:]
            while (len(binary) < wires):
                binary = "0" + binary
            tuple = (val, binary)
            adjust.append(tuple)
        else:
            adjust.append(line)
    return adjust
def cleanstate(numwires, myState):
    wires = int(numwires)
    vector = np.zeros(2**wires, dtype = "complex")
    for state in myState:
        decimal = int(int(state[1], 2))
        vector[decimal] += state[0]
    return VecToState(vector)
def cFuncIIComp(controlWire, firstWire, numWires, func, x, N, repeats, totalwires, inputState):
    controlwire = int(controlWire)
    numwires = int(numWires)
    firstwire = int(firstWire)
    #func only has xymodn so ignore it
    repeats = int(repeats)
    x = int(x)
    N = int(N)
    x = int(pow(x, repeats, N)) #use x^n mod n = x mod n 

    totalwires = int(totalwires)
    adjust = []
    offset = 2**firstwire
    for line in inputState:
        num = int(int(line[1],2)) 
        adjustedNum = int(num/offset)
        shift = num % offset
        if (adjustedNum < N and adjustedNum < 2**numwires and (num % 2**(controlwire+1) >= 2**(controlwire))):
            newnum = (adjustedNum *x) 
            val = line[0]
            actual = (newnum*offset + shift) % N
            binary = bin(actual)[2:]
            while (len(binary) < totalwires):
                binary = "0" + binary
            tuple = (val, binary)
            adjust.append(tuple)
        else: #if out of bounds just return 
            adjust.append(line)
    return adjust

In [35]:
def ParseInfoII(circuitfile):
    file = open(circuitfile, "r")
    lines = file.readlines()
    lineCount = 0
    wireCount = 0
    for i in range(len(lines)):
        if i == 0:
            wireCount = int(lines[i])
            myVector = np.zeros(2**wireCount, dtype = "complex") #placeholder but in case nothing is input
            myVector[0] = 1
            state = VecToState(myVector)
        else:
            line = lines[i]
            parts = line.split()
            if parts[0] == "H":
                state = HadamardII(parts[1], wireCount, state)
                state = cleanstate(wireCount, state)
            elif parts[0] == "P":
                state = PII(parts[1], parts[2], wireCount,state)
            elif parts[0] == "CNOT":
                state = CNOTII(parts[1], parts[2], wireCount, state)
            elif parts[0] == "CFUNC":
                state = cFuncIIComp(parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7], wireCount, state)
            elif lines[i][:1] == "I": # this entire section is if a state is input
                if lines[i][10:11] == "F":
                    myVector = getVector (lines[i][15:-1], wireCount) #-1 for pesky /n
                    state = VecToState(myVector)
                elif lines[i][10:11] == "B":
                    state = [lines[i][17:-1]] #-1 to snip >           
            elif lines[i][:1] == "M":
                #prob is prop to abs val of magnitudea
                myVector = StateToVec(state)
                adVector = abs(myVector)
                adVector = adVector**2
                myVector = np.zeros(2**wireCount)
                for j in range(500):
                    val = random.random()
                    i = 0
                    while (val - adVector[i] > 0):
                        val = val - adVector[i]
                        i += 1
                    myVector[i] += 1
                return myVector
    file.close()
    return StateToVec(state)

In [36]:
def Not(wireNum):
    out = ""
    out += "H " + str(wireNum) + "\n"
    out += "P " + str(wireNum) + " " + str(np.pi)  +"\n" #flips the |1> portion
    out += "H " + str(wireNum) + "\n"
    return out
def Rz(wireNum, phase):
    rot = float(phase)/2
    negrot = rot * -1
    rot = negrot * -1
    out = ""
    out = NOT(wireNum) #flips |0> and |1>
    out += "P " + str(wireNum) + " " + str(negrot) + "\n" #adjust |0> portion
    out += NOT(wireNum)
    out += "P " + str(wireNum) + " " + str(rot) + "\n" #adjust |1> portion
    return out
def CRz(other, control, phase):
    rot = float(phase)/2
    negrot = rot * -1
    rot = negrot * -1
    out = ""
    out = "CNOT " + str(control) + " " + str(other) + "\n"
    out += "P " + str(other) + " " +str(negrot) + "\n"
    out += "CNOT " + str(control) + " " + str(other) + "\n"
    out += "P " + str(other) + " " + str(rot) + "\n"
    return out
def CP(first, second, phase):
    #wrong
    rot = float(phase)/2
    negrot = rot * -1
    rot = negrot * -1
    out = ""
    out = "P " + str(first) + " " + str(rot)  + "\n"
    out += "P " + str(second) + " " + str(rot)  + "\n"
    out += "CNOT " + str(first) + " " + str(second) + "\n"
    out += "P " + str(second) + " " + str(negrot)  + "\n"
    out += "CNOT " + str(first) + " " + str(second) + "\n"
    return out
def singleSwap(first, second):
    out = ""
    out += "CNOT " + str(second) + " " + str(first) + "\n"
    out += "CNOT " + str(first) + " " + str(second) + "\n"
    out += "CNOT " + str(second) + " " + str(first) + "\n"
    return out
def Swap(first, second):
    first = int(first)
    second = int(second)
    out = ""
    if (first < second):
        larger = second
        smaller = first
    else:
        larger = first
        smaller = second
    largerhold = smaller
    while (larger > smaller):
        out += singleSwap(larger - 1, larger)
        larger += -1
    smaller += 1
    while (smaller < largerhold):
        out += singleSwap(smaller, smaller+1)
        smaller += 1
    return out

In [37]:
def first(numTopWires):
    out = ""
    for i in range(numTopWires):
        out += "H " + str(i) + "\n"
    return out
def QFT(numTopWires):
    out = ""
    startphase = np.pi
    for i in range(numTopWires):
        for j in range(i - 1, -1, -1):
            phase *= 1/2
            out += "CPHASE " + str(i) + " " + str(j) + " " + str(phase) + "\n"
        out += "H " + str(i) + "\n" # initial hadamard
        phase = -startphase
    return out
def reverse(numTopWires):
    out = ""
    a = 0 
    b = numTopWires - 1
    while (a < b):
        out += "SWAP " + str(a) + " " + str(b) + "\n"
        a += 1
        b += -1
    return out
def PreComp(file):
    file = open(file, "r")
    lines = file.readlines()
    lineCount = 0
    wireCount = 0
    out = ""
    for i in range(len(lines)):
        line = lines[i]
        parts = line.split()
        if (i == 0):
            out += lines[0] #first line is left as is
        elif (parts[0] == "NOT"):
            out += NOT(parts[1])
        elif (parts[0] == "RZ"):
            out += Rz(parts[1], parts[2])
        elif (parts[0] == "CRZ"):
            out += CRz(parts[1], parts[2], parts[3])
        elif (parts[0] == "CPHASE"):
            out += CP(parts[1], parts[2], parts[3])
        elif (parts[0] == "SWAP"):
            out += Swap(parts[1], parts[2])
        else: #if one of the previous quantum handled things just leave as is
            out += line
        #do stuff
    file.close()
    return out

In [38]:
def secondxyModNComp(numTopWires, numBottomWires, x, N):
    out = ""
    T = numTopWires
    B = numBottomWires
    for i in range(T):
        for k in range(B):
            out += "CFUNC " + str(T - 1 - i) + " " + str(T) + " " + str(B) + " xyModN " + str(x) + " " + str(N) + " " + str(2**i) + " "+ str(T+B) + "\n"
    return out

In [39]:
def PhaseEstimation(numTopWires, numBottomWires, x, N):
    file = open("qftstate.txt", "w")
    out2 = "1 1" #input |0001>
    file.write(out2)
    file.close()
    out = str(numTopWires + numBottomWires) + "\n"
    out += "INITSTATE FILE qftstate.txt\n"
    out += first(numTopWires)
    out += secondxyModNComp(numTopWires, numBottomWires, x, N)
    out += QFT(numTopWires)
    out += reverse(numTopWires)
    return out

In [40]:
def findRQuantum(N,x):
    numTopWires = 2* math.ceil(math.log2(N))+1
    numBottomWires = math.ceil(math.log2(N))
    file1 = open("preQFT.txt", "w")
    file1.write(PhaseEstimation(numTopWires, numBottomWires, x, N))
    file1.close()
    file2 = open("QFT.txt", "w")
    file2.write(PreComp("preQFT.txt"))
    file2.close()
    b = ParseInfoII("QFT.txt")
    adVector = abs(b)
    adVector = adVector**2
    b = np.zeros(2**numTopWires)
    highest = 0
    index = 0
    for i in range(len(adVector)):
        if (adVector[i] > highest):
            highest = adVector[i]
            index = i
    myDecimal = (index % 2**numTopWires)/2**numTopWires
    r= (Fraction(myDecimal).limit_denominator(N)).denominator
    return r

In [41]:
def Shors(N):
    factors = []
    if nt.isprime(N): #https://www.pythonpool.com/check-if-number-is-prime-in-python/ was used
        factors.append(N)
        return factors
    numtwos = factorTwo(N)
    for i in range(numtwos):
        factors.append(2)
        N *= (1/2)
    N = int(N)
    if N == 1:
        return factors
    base = testPowers(N)
    if base != 1:
        power = int(math.log(N,base))
        for i in range(power):
            factors.append(base)
        return factors
    repeat = True
    count = 0 
    while repeat and count < 10:
        x = bashgcd(N)
        r = findRQuantum(N,x)
        if (r % 2 == 0):
            count += 1
            minus = int(pow(x, int(r/2), N)) - 1
            plus = minus + 2
            minus = math.gcd(minus, N)
            plus = math.gcd(plus,N)
            if minus != 1 and minus != N:
                if (nt.isprime(minus)):
                    factors.append(minus)
                else:
                    factors = factors + Shors(minus)
                N = int(N/minus)
                if (nt.isprime(plus)):
                    factors.append(plus)
                else:
                    factors = factors + Shors(plus)
                N = int(N/plus)
                factors = factors + Shors(N)
                repeat = False
    return factors

In [48]:
print(Shors(21))

[3, 7]
